In [53]:
import spacy
import json
import os
from nltk.corpus import stopwords
import pandas as pd
import gensim
import gensim.corpora as corpora
import collections

In [ ]:
# files = []
# subdir = tuple(os.walk("../imdb/imdb_movie_jsons/1990"))
# for file_name in tuple(subdir)[0][-1]:
#     with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
#         files.append(json.load(f))

In [ ]:
# ner = spacy.load("en_core_web_sm")
# test = "Speed was a surprise hit movie. While it cost about $30,000,000 to make, it made back more than four times the cost. So, some Hollywood executives (with a collective IQ similar to that of a gerbil) thought...'if we do a sequel, we must spend $120,000,000 on it because THAT is why the first movie was such a success'. Not surprisingly, the movie tanked and it somehow has made it to IMDB's infamous Bottom 100 list the 100 worst ranked major releases of all time.In the original, Keanu Reeves and Sandra Bullock work to prevent a bus from exploding and killing folks. Now, because this is an overbloated sequel, they've placed the story aboard a cruise ship...which is also out of control and will kill everyone unless our heroes save it....though this time Reeves refused to reprise his role. So, they cast the rather bland Jason Patric in the lead...who offered little improvement other than he was very good at using sign language in one small scene.How bad is this film? Well, not bad enough to be on the Bottom 100 list. So why is it on the list? After loving the first film, so many folks expected more of the same...and got nothing but a dull overblown action picture with little in the way to interest viewers....and I think THAT is why it made the list....because it disappointed so badly. Instead, I wish IMDB had a list for films with the worst ratio of cost to enjoyability of the film...in which case \"Speed 2\" would easily make the top 100.By the way, this sequel ended up spending nearly as much for ONE crash scene as the budget for all of \"Speed\"! And, after paying $11,500,000 just to obtain Sandra Bullock for this film, you can only assume that the film simply had nothing left to pay competent writers. Instead, I assume, they hired some penguins to write the script....some very, very dull and untalented penguins!"
# doc = ner(test)
# for ent in doc.ents:
#     print(f"{ent.text} {ent.label} {str (spacy.explain(ent.label_))}")

In [ ]:

with open("../stopwords_list.txt", 'r') as f:
    vibes = f.readlines()

In [ ]:
nlp = spacy.load("en_core_web_sm")
corpus = []
for vibe in vibes:
    vibe_per_movie = nlp(vibe)
    clean = []
    for word in vibe_per_movie:
        if word.pos_ == "ADJ":
            clean.append(str(word.lemma_))
    corpus.append(" ".join(clean).strip())


In [ ]:
corpus

In [ ]:
corpus = [s for s in corpus if len(s) > 3]
corpus

In [ ]:
proc_data = [string.split() for string in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [ ]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict)
vector = model[input_corpus]

In [ ]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f

In [ ]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

In [55]:
a = []
for word in corpus:
    a.extend(word.split())
# print(words)
a = dict(collections.Counter(a))
with open("../stopwords.json", "w") as stopwords:
    json.dump([k for k,v in a.items() if v >= 1125], stopwords)